In [182]:
import pandas as pd
from random import randint
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from ipywidgets import interact, Layout
import plotly.io as pio
import math
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from typing import Callable
import datetime

# Données et fonctions

#### Dictionnaire des variables essentielles - explication des variables

Ce dictionnaire contient l'ensemble des variables utilisées dans le code, à l'exception des variables de la dernière partie, qui concerne la surface du Data Center. Cette dernière partie comprend un dictionnaire distinct avec les valeurs techniques spécifiques.

- `nb_months_experiment` : nombre total de mois pendant lesquels l'expérience est menée (type : int)
  
- `nb_gpu_regroupement` : nombre de GPU par regroupement (type : int)
- `random_nb_gpu_to_multiply` : liste pour simuler une plage de valeurs potentielles (aléatoires) pour le nombre de GPU à multiplier ensuite par `nb_gpu_regroupement` (type : list[int])
- `lifespan_months` : durée de vie des GPU en mois (type : int)
- `multiplication_of_demand` : facteur par lequel la demande de FLOPS est multipliée après une certaine période (type : float)
- `duration_between_demand_multiplications` : durée entre chaque multiplication de la demande en mois (type : int)
- `usage_rate` : taux d'utilisation des GPU dans le DataCenter, la valeur est comprise entre 0 et 1 (type : float)
- `performance_equipment` : performance supplémentaire liée aux équipements de support comme la carte mère, le CPU, etc. en Watts (type : int)
- `gpu_performance_static` : performance statique du GPU utilisée pour des calculs de performance génériques en Watts (type : int)
- `hours_of_use_per_day` : nombre d'heures pendant lesquelles les GPU sont utilisés par jour (type : int)
- `days_of_use_per_month` : nombre de jours pendant lesquels les GPU sont utilisés par an (type : int)
- `alpha` : paramètre alpha utilisé dans les calculs de modélisation, la valeur est comprise entre 0 et 1 (type : float)
- `beta` : paramètre beta utilisé dans les calculs de modélisation, la valeur doit être positive (type : float)
- `current_year` : année courante pour laquelle l'expérience commence (type : int)
- `current_month` : mois courant pour lequel l'expérience commence (type : int)

In [183]:
dict_variables = {'nb_months_experiment' : 84,
                 'nb_gpu_regroupement' : 8,
                 'random_nb_gpu_to_multiply' : [1_000, 1_100],
                 'Lifespan_Months' : 36,
                 'multiplication_of_demand' : 2.5,
                 'duration_between_demand_multiplications' : 30,
                 'usage_rate' : 0.8,
                 'performance_equipment' : 66,
                 'gpu_performance_static' : 25,
                 'hours_of_use_per_day' : 24, 
                 'days_of_use_per_month' : 30,
                 'alpha' : 0.5,
                 'beta' : 1,
                 'current_year' : 2019,
                 'current_month' : 1,
                 }

____

## Lecture des fichiers CSV contenant les données 

Tous ces fichiers se trouvent dans le dossier `csv_files`, situé dans le répertoire `data` de ce projet. Si nécessaire, les valeurs des données peuvent être modifiées manuellement en accédant au fichier `Data_GPU.ipynb`, qui se trouve également dans le répertoire `data`. Les fichiers contiennent les informations suivantes :

#### Table de l’efficatité du matériel disponible :

In [184]:
df_data_GPU = pd.read_csv('data/csv_files/data_GPU.csv')
df_data_GPU

,GPU_Name,Performance_Tensor,Max_Power,Price,Release_Date,CO2_Emissions_Production,Water_Consumption_Production,Abiotic_Depletion_Production,Electronic_Waste_Production
0,V100,125,300,26300,2019-01,100,50,0.0150,1.60
1,A100,500,400,31900,2020-06,150,75,0.0220,1.92
2,H100,2000,700,29500,2023-01,150,100,0.0285,1.92


#### Liste des émissions de CO2 liée à la consommation en kgCO2e/kWh par année

In [185]:
df_co2_emissions_per_kwh = pd.read_csv('data/csv_files/co2_emissions_per_kWh.csv')
df_co2_emissions_per_kwh.head()

,Year,CO2_per_kWh
0,2019,0.061000
1,2020,0.060000
2,2021,0.057000
3,2022,0.052000
4,2023,0.050615


#### Liste de la consommation d'eau liée à l'utilisation électrique en m³ par année

In [186]:
df_water_consumption_per_kwh = pd.read_csv('data/csv_files/water_consumption_per_kWh.csv')
df_water_consumption_per_kwh.head()

,Year,Water_per_kWh
0,2019,0.0033
1,2020,0.0033
2,2021,0.0033
3,2022,0.0033
4,2023,0.0033


#### Liste de la consommation des ressources abiotiques en kg/Sbeq par année 

In [187]:
df_abiotic_depletion_per_kwh = pd.read_csv('data/csv_files/abiotic_depletion_per_kWh.csv')
df_abiotic_depletion_per_kwh.head()

,Year,Abiotic_Depletion_per_kWh
0,2019,4.860000e-08
1,2020,4.860000e-08
2,2021,4.860000e-08
3,2022,4.860000e-08
4,2023,4.860000e-08


#### La liste des quantités des déchets électroniques en kg par année 

In [188]:
df_electronic_waste_quantities = pd.read_csv('data/csv_files/electronic_waste_quantities.csv')
df_electronic_waste_quantities.head()

,Year,Electronic_Waste_Quantities
0,2019,0
1,2020,0
2,2021,0
3,2022,0
4,2023,0


___

## Dictionnaire des Indicateurs 

Ce dictionnaire contient les indicateurs utilisés pour analyser l'impact environnemental du Data Center. Chaque indicateur inclut les informations suivantes :

- **Nom de l'indicateur** : le nom de l'indicateur environnemental, utilisé comme clé dans le dictionnaire

- Les valeurs associées à chaque clé se trouvent dans une liste comprenant :

  - **Fichier CSV** : le fichier CSV contenant les données relatives à cet indicateur. Ces fichiers doivent être lus au préalable, comme décrit dans la section "Lecture des fichiers CSV"
    
  - **Nom de la variable** : le nom de la variable dans le fichier CSV qui contient les données sur la quantité de l'indicateur par unité, exprimée par année, jusqu'en 2035
  - **Unité** : l'unité de mesure de l'indicateur
  - **Quantité utilisée pour la production d'un serveur** : la quantité de l'indicateur utilisée pour la production d'un serveur, exprimée dans l'unité correspondante

Le dictionnaire contient actuellement les indicateurs suivants :

- **Emissions CO2** : Quantité de CO2 émise par kg
  
- **Water Consumption** : Consommation d'eau par m³
- **Abiotic Depletion** : Déplétion abiotique par kg/Sbeq
- **Electronic Waste** : Quantité de déchets électroniques par kg

In [189]:
dict_indicators = {'CO2_Emissions' : [df_co2_emissions_per_kwh, 'CO2_per_kWh', 'kg', 150],
                  'Water_Consumption' : [df_water_consumption_per_kwh, 'Water_per_kWh', 'm³', 3500],
                  'Abiotic_Depletion' : [df_abiotic_depletion_per_kwh, 'Abiotic_Depletion_per_kWh', 'kg/Sbeq', 0.16],
                  'Electronic_Waste' : [df_electronic_waste_quantities, 'Electronic_Waste_Quantities', 'kg', 10.7]
                  }

___

## Configuration des différentes Listes et DataFrames 

### Création du DataFrame pour le matériel initial du parc de GPUs

Cette fonction crée un DataFrame avec des détails sur le matériel du parc de GPUs, incluant le nombre total de GPUs, leur durée de vie, et le nom du GPU, en utilisant des paramètres aléatoires et définis.

In [190]:
def create_df_gpu_park(df_data_GPU: pd.DataFrame, dict_variables: dict[str, int]) -> pd.DataFrame:    
    random_nb_gpu_to_multiply = dict_variables.get('random_nb_gpu_to_multiply')
    nb_gpu_regroupement = dict_variables.get('nb_gpu_regroupement')
    random_nb_gpu_to_multiply = dict_variables.get('random_nb_gpu_to_multiply')
    list_GPU_park = []
    
    nb_GPU = randint(random_nb_gpu_to_multiply[0], random_nb_gpu_to_multiply[1]) * nb_gpu_regroupement
    nb_months_experiment_renew = 0
    gpu_name = df_data_GPU['GPU_Name'].values[0]
    list_GPU_park.append((nb_GPU, nb_months_experiment_renew, gpu_name))
    return pd.DataFrame(list_GPU_park, columns = ['Total_GPU_Count', 'Lifespan_Months', 'GPU_Name'])

### Liste des FLOPS souhaités dans le parc par an

La fonction génère une liste des FLOPS requis pour chaque mois de l'expérience, en tenant compte des périodes de multiplication de la demande.

In [191]:
def create_list_flops_per_year(df_GPU_park: pd.DataFrame, df_data_GPU: pd.DataFrame, dict_variables: dict[str, int], duration_between_demand_multiplications: int) -> list[int]:
    nb_months_experiment = dict_variables.get('nb_months_experiment')
    multiplication_of_demand = dict_variables.get('multiplication_of_demand')
    list_flops_required = []
    flop_required_year_1 = int(df_GPU_park['Total_GPU_Count'][0] * df_data_GPU['Performance_Tensor'].iloc[0])
    list_flops_required.append(flop_required_year_1)
    for i in range(1, nb_months_experiment):
        if i % duration_between_demand_multiplications == 0:
            flop_year = list_flops_required[i-1] * multiplication_of_demand
        else:
            flop_year = list_flops_required[i-1]
        list_flops_required.append(flop_year)
    return list_flops_required

### Liste des taux d'utilisation des GPU

Permet de génère une liste des taux d'utilisation des GPU pour chaque mois de l'expérience, en utilisant une valeur fixe définie.

In [192]:
def create_list_usage_rate(dict_variables: dict[str, int]) -> list[float]:
    list_flops_required = []
    
    nb_months_experiment = dict_variables.get('nb_months_experiment')
    usage_rate = dict_variables.get('usage_rate')
    
    for _ in range(nb_months_experiment):
        list_flops_required.append(usage_rate)
    return list_flops_required

___

## Extension du DataFrame des GPU avec des colonnes supplémentaires 

### Taux d'usage

La fonction ajoute une colonne `Usage_Rate` au DataFrame `df_GPU_park`, avec une valeur fixe `usage_rate` pour chaque entrée.

In [193]:
def add_column_usage_rate(df_GPU_park: pd.DataFrame, usage_rate: float) -> pd.DataFrame:
    df_GPU_park['Usage_Rate'] = usage_rate
    return df_GPU_park

### Consommation énérgetique

Cette fonction ajoute une colonne `Energy_Consumption` au DataFrame `df_GPU_park`, calculant la consommation énergétique basée sur les performances des GPU, le taux d'utilisation, et les paramètres du système. Les informations sur la puissance des GPU sont obtenues à partir du DataFrame `df_data_GPU`.

In [194]:
def add_column_energy_consumption(df_GPU_park: pd.DataFrame, df_data_GPU: pd.DataFrame, dict_variables: dict[str, int]) -> pd.DataFrame:
    performance_equipment = dict_variables.get('performance_equipment')
    gpu_performance_static = dict_variables.get('gpu_performance_static')
    hours_of_use_per_day = dict_variables.get('hours_of_use_per_day')
    days_of_use_per_month = dict_variables.get('days_of_use_per_month')
    
    merged_df = pd.merge(df_GPU_park, df_data_GPU[['GPU_Name', 'Max_Power']], on='GPU_Name')
    merged_df['Energy_Consumption'] = (merged_df['Total_GPU_Count'] * hours_of_use_per_day * days_of_use_per_month * ((merged_df['Max_Power'] + performance_equipment + gpu_performance_static) * merged_df['Usage_Rate'] + (1 - merged_df['Usage_Rate']) * gpu_performance_static) / 100).astype(int)
    return merged_df.drop("Max_Power", axis=1)

### Performance - TFLOPS disponibles et utilisés des GPU

La fonction ajoute deux colonnes, `TFlop_Available` et `TFlop_Used`, au DataFrame `df_GPU_park`. Elle calcule les FLOPS disponibles et utilisés en fonction de la performance des GPU et du taux d'utilisation, en se basant sur les données fournies dans `df_data_GPU`.

In [195]:
def add_column_TFlops(df_GPU_park: pd.DataFrame, df_data_GPU: pd.DataFrame) -> pd.DataFrame:
    merged_df = pd.merge(df_GPU_park, df_data_GPU[['GPU_Name', 'Performance_Tensor']], on='GPU_Name')
    merged_df['TFlop_Available'] = merged_df['Total_GPU_Count'] * merged_df['Performance_Tensor']
    merged_df['TFlop_Used'] = merged_df['TFlop_Available'] * merged_df['Usage_Rate']
    return merged_df.drop('Performance_Tensor', axis=1)

### Coût total des GPU ajoutés au DataFrame

Ajoute une colonne `Price_GPU_Added` au DataFrame `df_GPU_park`. La fonction calcule le coût total des GPU ajoutés en fonction de leur prix et de leur nombre, mais uniquement si la durée de vie des GPU (`Lifespan_Months`) est nulle, c'est-à-dire que les GPU viennent d'être ajoutés au parc. Les prix sont extraits de `df_data_GPU`, et la colonne `Price` est supprimée après le calcul.

In [196]:
def add_column_price_gpu(df_GPU_park: pd.DataFrame, df_data_GPU: pd.DataFrame) -> pd.DataFrame:
    merged_df = pd.merge(df_GPU_park, df_data_GPU[['GPU_Name', 'Price']], on='GPU_Name')
    merged_df['Price_GPU_Added'] = 0
    for i in range(merged_df.shape[0]):
        if merged_df['Lifespan_Months'].iloc[i] == 0:
            merged_df.loc[i, "Price_GPU_Added"] = merged_df['Total_GPU_Count'].iloc[i] * merged_df['Price'].iloc[i]
    return merged_df.drop('Price', axis=1)

### Ajout des colonnes pour les indicateurs définis dans le dictionnaire `dict_indicators`

#### Calcul de la consommation totale de ressources pour les indicateurs de production de GPU

Calcule la consommation totale de ressources lors de la production de GPU, en fonction des types et du nombre de GPU ainsi que de la consommation associée à la production des serveurs. La fonction utilise les informations du DataFrame `df_data_GPU` et les valeurs spécifiées dans une ligne du DataFrame `row`, qui est associé à un DataFrame contenant une seule ligne (la fonction sera appelée pour chaque ligne d'un DataFrame en utilisant une boucle `for`). Elle prend en compte le coût de production par serveur ainsi que le nombre de GPU regroupés.

In [197]:
def resource_consumption_gpu_production(df_data_GPU: pd.DataFrame, row: int, column: int, nb_gpu_regroupement: int, consumption_prod_server: float, column_df_data_GPU: str) -> float:
    total_consumption = 0
    if row[column] > 0:
        for key, val in row['GPU_Types'].items():
            df_consumption = df_data_GPU[df_data_GPU['GPU_Name'] == key][column_df_data_GPU].values
            if df_consumption.size > 0:
                total_consumption += row[column] * df_consumption[0] + math.ceil(row[column] / nb_gpu_regroupement) * consumption_prod_server
    return total_consumption

#### Ajout d'une colonne calculée basée sur les données annuelles pour les indicateurs :

La fonction ajoute une colonne à un DataFrame (`df_test_model`) en calculant une valeur dérivée des données d'un autre DataFrame (`df_data_per_year`). La colonne ajoutée est calculée en multipliant l'énergie consommée par une valeur spécifique à chaque année, extraite de `df_data_per_year`. La colonne de données annuelles et l'année utilisée pour le calcul sont supprimées du DataFrame final.

In [198]:
def add_column_usage(df_test_model: pd.DataFrame, df_data_per_year: pd.DataFrame, current_year: int, column_to_add: str, column_df_data_per_year: str) -> pd.DataFrame:
    df_test_model['Year'] = current_year
    merged_df = pd.merge(df_test_model, df_data_per_year[['Year', column_df_data_per_year]], on='Year')
    merged_df[column_to_add] = (merged_df['Energy_Consumption'] * merged_df[column_df_data_per_year]).astype(int)
    return merged_df.drop([column_df_data_per_year, 'Year'], axis=1)

___

## Calcul des données statistiques diverses

### Calcul de la Somme Totale des Données d'une Variable

Cette fonction calcule la somme totale d'une colonne spécifique dans le DataFrame `df_GPU_park`. La colonne est identifiée par `variable_name`, et le type de données est précisé par `data_type`. La fonction renvoie la somme sous le type de données spécifié, soit `int` ou `float`.

La fonction sera utilisé pour les données suivantes :
- **Nombre total de GPU dans le Data Center**
- **Consommation totale des GPU dans le Data Center en kWh**
- **Total des TFLOPS disponibles dans le parc**
- **Quantité totale de CO2 émise lors de l'utilisation des GPU en kg**

In [199]:
def calculate_total_sum_variable(df_GPU_park: pd.DataFrame, variable_name: str, data_type: type) -> float:
    total_sum = df_GPU_park[variable_name].sum()
    if data_type == int:
        return int(total_sum)
    elif data_type == float:
        return total_sum

### Détection des pics de demande de TFLOPS

Identifie les pics significatifs dans la demande de TFLOPS en comparant les valeurs dans `list_flops_required` avec un facteur de multiplication spécifié. Elle retourne une liste de pics permet de repérer les moments où des ajustements ou des ajouts de GPU peuvent être nécessaires pour répondre à la demande croissante.

In [200]:
def identify_demand_peaks(list_flops_required: list[float], dict_variables: dict[str, float]) -> list[list[float]]:
    list_demand_peacks = []
    multiplication_of_demand = dict_variables.get('multiplication_of_demand')
    
    for i in range(1, len(list_flops_required)):
        if list_flops_required[i] == list_flops_required[i-1] * multiplication_of_demand:
            list_demand_peacks.append([list_flops_required[i-1], list_flops_required[i], i])
    return list_demand_peacks

___

## Mise à jour et gestion du Data Center

### Mise à jour de la durée de vie restante des GPU (en mois)

La fonction `update_lifespan_gpu` met à jour la durée de vie restante des GPU dans le DataFrame `df_GPU_park`. Elle ajoute le nombre de mois écoulés (`months_passed`) à la colonne `Lifespan_Months`, ce qui reflète l'augmentation de la durée de vie totale des GPU. Cette mise à jour est essentielle pour suivre l'évolution de la durée de vie des GPU au fil du temps dans le parc informatique.

In [201]:
def update_lifespan_gpu(df_GPU_park: pd.DataFrame, months_passed: int) -> pd.DataFrame:
    df_GPU_park.loc[:, 'Lifespan_Months'] += months_passed
    return df_GPU_park

### Calcul du nombre de GPU nécessaires pour atteindre les exigences de performance du Data Center

Calcule le nombre de GPU nécessaires pour satisfaire une demande de FLOPS en fonction des performances maximales des GPU et des TFLOPS déjà disponibles dans le parc. La fonction prend en compte les TFLOPS à renouveler et ajuste les besoins en GPU en conséquence. Elle retourne le nombre total de GPU requis et le nombre de GPU à renouveler.

In [202]:
def how_many_gpu_needed(dict_variables: dict[str, int], flops_required: int, max_perf_gpu: int, total_flops_park: float, flops_to_renewed: float) -> tuple[int, int, int]:
    nb_gpu_regroupement = dict_variables.get('nb_gpu_regroupement')
    
    def round_up_to_multiple(n: int) -> int:
        return ((n + nb_gpu_regroupement - 1) // nb_gpu_regroupement) * nb_gpu_regroupement
    flops_needed = flops_required - total_flops_park
    if flops_to_renewed > flops_needed and flops_to_renewed != 0:
        flops_to_renewed = flops_needed
    if flops_required < total_flops_park:
        return 0, 0
    elif flops_needed > 0:
        required_gpus = flops_needed // (max_perf_gpu) + (flops_needed % max_perf_gpu > 0)
        required_gpus = round_up_to_multiple(required_gpus)
        gpu_renewed = round_up_to_multiple(flops_to_renewed // (max_perf_gpu) + (flops_to_renewed % max_perf_gpu > 0))
        return int(required_gpus), int(gpu_renewed)
    else:
        return 0, 0

#### Ajout des nouveaux GPU dans le Data Center

Ajoute de nouveaux GPU au parc existant, en mettant à jour les données du DataFrame `df_GPU_park`. Elle calcule la consommation énergétique, la performance en TFLOPS et le coût des nouveaux GPU, et met à jour les colonnes correspondantes. Les valeurs des indicateurs environnementaux (comme les émissions de CO2 et la consommation d'eau) sont également ajoutées en fonction des données de l'année courante. La fonction retourne le DataFrame mis à jour avec les nouveaux GPU intégrés.

In [203]:
def add_gpu_to_park(df_GPU_park: pd.DataFrame, dict_variables: dict[str, int], dict_indicators: dict[str, int], nb_gpu_to_add: int, max_power_gpu: int, max_performance_gpu: int, price_gpu: int, usage_rate: float, type_gpu: str, current_year: int) -> pd.DataFrame:
    performance_equipment = dict_variables['performance_equipment']
    gpu_performance_static = dict_variables['gpu_performance_static']
    hours_of_use_per_day = dict_variables['hours_of_use_per_day']
    days_of_use_per_month = dict_variables['days_of_use_per_month']
    
    if nb_gpu_to_add != 0:
        energy_consumption = (nb_gpu_to_add * (max_power_gpu + performance_equipment + gpu_performance_static) * usage_rate * 
                              (1 - usage_rate) * gpu_performance_static * (hours_of_use_per_day * days_of_use_per_month) / 1000)
        
        new_gpu_data = {
            'Lifespan_Months': 0,
            'Total_GPU_Count': nb_gpu_to_add,
            'GPU_Name': type_gpu,
            'Usage_Rate': usage_rate,
            'Energy_Consumption': energy_consumption,
            'TFlop_Used': nb_gpu_to_add * max_performance_gpu * usage_rate,
            'TFlop_Available': nb_gpu_to_add * max_performance_gpu,
            'Price_GPU_Added' : nb_gpu_to_add * price_gpu
        }

        for indicator, (df_indicator, column_name, unity, value_prod_server) in dict_indicators.items():
            quantity_consumption = df_indicator.query('Year == @current_year')[column_name].values[0]
            quantity_consumption_usage = energy_consumption * quantity_consumption
            new_gpu_data[f'{indicator}_Usage'] = round(quantity_consumption_usage, 2) 
        
        df_GPU_park = pd.concat([df_GPU_park, pd.DataFrame([new_gpu_data])], ignore_index=True)
    
    return df_GPU_park

### Identifie le GPU le plus récent disponible à la date spécifiée

Détermine le type de GPU le plus récent disponible jusqu'à une date donnée (année et mois). Elle filtre les GPU dont la date de sortie est antérieure ou égale à la date spécifiée, puis sélectionne le modèle avec la date de sortie la plus récente. La fonction retourne les caractéristiques du GPU le plus récent : nom du modèle, puissance maximale, performance en TFLOPS, et prix.

In [204]:
def most_recent_gpu(df_data_GPU: pd.DataFrame, current_year: int, current_month: int) -> tuple[str, int, int, int]:
    current_year_str = str(current_year)
    current_month_str = str(current_month)

    current_date = current_year_str + '-0' + current_month_str if current_month<10 else current_year_str + '-' + current_month_str
    df_data_GPU['Release_Date'] = pd.to_datetime(df_data_GPU['Release_Date'], format='%Y-%m')

    df_filtered = df_data_GPU[df_data_GPU['Release_Date'] <= current_date]
    most_recent_gpu = df_filtered.loc[df_filtered['Release_Date'].idxmax()]
    
    type_gpu = most_recent_gpu['GPU_Name']
    max_power_gpu = most_recent_gpu['Max_Power']
    max_flop_gpu = most_recent_gpu['Performance_Tensor']
    price_gpu = most_recent_gpu['Price']
    
    return type_gpu, max_power_gpu, max_flop_gpu, price_gpu

### Retrait des GPU dont la durée de vie dépasse une période spécifiée

Supprime les GPU dont la durée de vie dépasse une période donnée (en mois) du DataFrame `df_GPU_park`. Elle calcule la somme totale des TFLOPS disponibles avant et après le retrait des GPU, et retourne le DataFrame mis à jour ainsi que la différence de TFLOPS disponibles. Les GPU retirés sont ceux dont la durée de vie est supérieure à la période spécifiée, et leur nombre total est également calculé.

In [205]:
def remove_gpu_lifespan(df_GPU_park: pd.DataFrame, lifespan_months: int) -> pd.DataFrame:
    available_flops_park_before = calculate_total_sum_variable(df_GPU_park, 'TFlop_Available', float)
    nb_gpu_removed = 0
    for i in range(len(df_GPU_park)):
        if df_GPU_park['Lifespan_Months'][i] > lifespan_months:
            nb_gpu_removed += df_GPU_park['Total_GPU_Count'][i]
            df_GPU_park = df_GPU_park.drop(i)
    available_flops_park_after = calculate_total_sum_variable(df_GPU_park, 'TFlop_Available', float)
    return df_GPU_park.reset_index(drop=True), int(available_flops_park_before - available_flops_park_after)

### DDictionnaire des types de GPU et de leurs quantités

Cette fonction crée un dictionnaire où chaque clé est le nom d'un type de GPU disponible dans le Data Center, et chaque valeur est la quantité totale de ce type de GPU. Elle parcourt le DataFrame `df_GPU_park`, agrège les quantités de GPU par type, et retourne le dictionnaire résultant.

In [206]:
def dict_type_gpu(df_GPU_park: pd.DataFrame) -> dict[str, int]:
    dict_gpu = {}
    for i in range(len(df_GPU_park)):
        name_gpu = df_GPU_park['GPU_Name'].iloc[i]
        quantity_gpu = int(df_GPU_park['Total_GPU_Count'].iloc[i])
        
        if name_gpu in dict_gpu:
            dict_gpu[name_gpu] += quantity_gpu
        else:
            dict_gpu[name_gpu] = quantity_gpu 
    
    return dict_gpu

___

# Modèle

## Initialisation des données du Data Center

La fonction `initialise_data` prépare le DataFrame du parc de GPU en créant d'abord un DataFrame avec les informations de base. Elle enrichit ensuite ce DataFrame avec des données telles que le taux d'utilisation, la consommation énergétique, les performances en TFLOPS et le coût des GPU. Enfin, elle ajoute des colonnes pour les indicateurs de consommation basés sur les données annuelles. Le DataFrame ainsi obtenu offre un état détaillé du parc de GPU au début de l'expérience, prêt pour l'analyse.

In [207]:
def initialise_data(dict_variables: dict[str, int], usage_rate: float) -> pd.DataFrame:
    current_year = dict_variables.get('current_year')
    df_GPU_park = create_df_gpu_park(df_data_GPU, dict_variables)
    df_GPU_park = add_column_usage_rate(df_GPU_park, usage_rate)
    df_GPU_park = add_column_energy_consumption(df_GPU_park, df_data_GPU, dict_variables)
    df_GPU_park = add_column_TFlops(df_GPU_park, df_data_GPU)
    df_GPU_park = add_column_price_gpu(df_GPU_park, df_data_GPU)
    for i in range(len(dict_indicators)):
        df_GPU_park = add_column_usage(df_GPU_park, list(dict_indicators.values())[i][0], current_year, f'{list(dict_indicators.keys())[i]}_Usage', list(dict_indicators.values())[i][1])
    return df_GPU_park

Le DataFrame suivant reflète l'état du parc de GPU au début de l'expérience avec toutes les informations calculées et ajoutées. Une copie est créée pour effectuer des modifications sans altérer les données originales.

In [208]:
usage_rate = dict_variables.get('usage_rate')
df_GPU_park_init = initialise_data(dict_variables, usage_rate)
df_GPU_park = df_GPU_park_init.copy() 
df_GPU_park

,Total_GPU_Count,Lifespan_Months,GPU_Name,Usage_Rate,Energy_Consumption,TFlop_Available,TFlop_Used,Price_GPU_Added,CO2_Emissions_Usage,Water_Consumption_Usage,Abiotic_Depletion_Usage,Electronic_Waste_Usage
0,8152,0,V100,0.8,18653080,1019000,815200.0,214397600,1137837,61555,0,0


Liste contenant le taux d'utilisation des GPU pour chaque mois de l'expérience

In [209]:
list_gpu_rate_usage = create_list_usage_rate(dict_variables)
list_gpu_rate_usage[0:5]

[0.8, 0.8, 0.8, 0.8, 0.8]

____

## Algorithme 



### Fonctions pour Simuler le Taux d'Utilisation des GPU

Pour notre algorithme, nous allons évaluer plusieurs fonctions pour simuler le taux d'utilisation des GPU ajoutés, afin d'atténuer les effets d'escalier observés lorsqu'un grand nombre de GPU est ajouté simultanément. 

La courbe lissée réduit les variations brusques des valeurs originales, offrant une vue plus stable de la tendance générale du taux d'utilisation des GPU. Les paramètres $\alpha$ ou $\beta$ ajuste la réactivité de la courbe lissée aux nouvelles données.

On cherche à estimer : $\bar{f} =\frac{\tau}{c_i}$ qui represente la valeur cible et on ne peut pas augmenter $f_i$ de plus de 20% (par exemple), avec $\tau$ à 80% et $\tau_{i-1}$ initié avec $\tau$ et $c_i$ la capacité (TFLOP) en fonction du temps.

Ainsi : $\tau_i = \frac{f_i}{c_i}$

**Première fonction :** lissage exponentiel

- $f_{1_i} = (1 - \alpha) \cdot f_{i-1} + \alpha \cdot \bar{f} \quad \text{avec} \quad \alpha \in (0, 1]$

In [210]:
def estimate_fi_1(f_prev: float, f_bar: float, alpha: float) -> float:
    f_i = (1 - alpha) * f_prev + alpha * f_bar
    return min(f_i, f_prev * 1.2)

**Deuxième fonction :** lissage avec limite

- $\begin{cases}
f_i = \min(\bar{f}, f_{i-1} + \Delta) \\
\Delta = f_{i-1} \cdot \beta \quad \text{avec} \quad \beta \in (0, \infty[
\end{cases}$

In [211]:
def estimate_fi_2(f_prev: float, f_bar: float, beta: float) -> float:
    delta = f_prev * beta
    return min(f_bar, f_prev + delta)

**Troisième fonction :** croissance proportionnelle

- $\Delta = \bar{f} \cdot \beta \text{ avec} \quad \beta \in (0, 1]$

In [212]:
def estimate_fi_3(f_prev: float, f_bar: float, beta: float) -> float:
    delta = f_bar * beta
    return min(f_bar, f_prev + delta)

### Modèle de simulation du Data Center de GPU pour une période (en mois)

La fonction `model` met à jour l'état du parc de GPU en fonction des besoins en performance et en consommation. Elle commence par actualiser la durée de vie des GPU et retirer ceux qui sont obsolètes. Ensuite, elle détermine le type de GPU le plus récent et calcule combien de GPU supplémentaires sont nécessaires pour atteindre les performances requises. Si des GPU doivent être ajoutés, elle les intègre au parc et ajuste les données en conséquence. La fonction met également à jour le taux d'utilisation des GPU si des données historiques sont disponibles, et calcule les totaux pour la consommation énergétique, les performances en TFLOPS, et le coût des GPU ajoutés. Le DataFrame mis à jour, ainsi que les totaux et les indicateurs sont renvoyés.

In [213]:
def model(df_data_GPU: pd.DataFrame, df_GPU_park: pd.DataFrame, dict_variables: dict[str, int], dict_indicators: dict[str, int], flops_required: int, lifespan_months: int, usage_rate: float, current_year: int, current_month: int, estimate_fi_func: Callable[[float, float, float], float], f_prev: float, f_bar: float, c_i: float, alpha: float, beta: float) -> tuple[pd.DataFrame, int, float, float, float, int, int, float, dict[str, int], dict[str, float], float, float]:
    tau = dict_variables.get('usage_rate')
    df_GPU_park = update_lifespan_gpu(df_GPU_park, 1)
    df_GPU_park, tflops_to_renewed = remove_gpu_lifespan(df_GPU_park, lifespan_months)
    type_gpu, max_power_gpu, max_performance_gpu, price_gpu = most_recent_gpu(df_data_GPU, current_year, current_month)
    dict_gpu = dict_type_gpu(df_GPU_park)
    
    available_flops = calculate_total_sum_variable(df_GPU_park, 'TFlop_Available', float)
    nb_gpu_to_add, nb_gpu_to_renewed = how_many_gpu_needed(dict_variables, flops_required, max_performance_gpu, available_flops, tflops_to_renewed)
    if nb_gpu_to_add != 0:
        df_GPU_park = add_gpu_to_park(df_GPU_park, dict_variables, dict_indicators, nb_gpu_to_add, max_power_gpu, max_performance_gpu, price_gpu, usage_rate, type_gpu, current_year)
        dict_gpu = dict_type_gpu(df_GPU_park)
    if f_prev != 0 and f_bar != 0 and c_i != 0:
        parameter = alpha if estimate_fi_func == estimate_fi_1 else beta
        f_prev = estimate_fi_func(f_prev, f_bar, parameter)
        tau = f_prev / c_i
    df_GPU_park['Usage_Rate'] = tau
    df_GPU_park = add_column_energy_consumption(df_GPU_park, df_data_GPU, dict_variables)
    df_GPU_park = add_column_TFlops(df_GPU_park, df_data_GPU)
    
    for i in range(len(dict_indicators)):
        df_GPU_park = add_column_usage(df_GPU_park, list(dict_indicators.values())[i][0], current_year, f'{list(dict_indicators.keys())[i]}_Usage', list(dict_indicators.values())[i][1])
    
    gpu_count = calculate_total_sum_variable(df_GPU_park, 'Total_GPU_Count', int)
    energy_consumption = calculate_total_sum_variable(df_GPU_park, 'Energy_Consumption', float)
    available_flops = calculate_total_sum_variable(df_GPU_park, 'TFlop_Available', float)
    used_flops = calculate_total_sum_variable(df_GPU_park, 'TFlop_Used', float)
    price_gpu = calculate_total_sum_variable(df_GPU_park, 'Price_GPU_Added', float)
    
    indicator_usage_totals = {f'{indicator}_Usage': calculate_total_sum_variable(df_GPU_park, f'{indicator}_Usage', float) for indicator in dict_indicators}

    return df_GPU_park, gpu_count, energy_consumption, available_flops, used_flops, nb_gpu_to_add, nb_gpu_to_renewed, price_gpu, dict_gpu, indicator_usage_totals, tau, f_prev

La fonction simule l'évolution d'un parc de GPU sur une période définie, en prenant en compte les besoins en TFLOPS et les caractéristiques des GPU. Un DataFrame mensuellement qui reflète les ajustements du parc, tels que l'ajout et le renouvellement de GPU, la consommation énergétique et les performances en TFLOPS. En intégrant les pics de demande et en utilisant des modèles pour estimer les variations de performance, la fonction compile les données essentielles pour une analyse détaillée de l'évolution du Data Center au cours de la période simulée.

In [214]:
def create_df_test_model(df_data_GPU: pd.DataFrame, df_GPU_park: pd.DataFrame, dict_variables: dict[str, int], dict_indicators: dict[str, int], duration_between_demand_multiplications: int, lifespan_months: int, 
                         list_gpu_rate_usage: list[float], tau: float, nb_months_experiment: int, estimate_fi_func: Callable[[float, float, float], float], current_year: int, current_month: int, alpha: float, beta: float) -> pd.DataFrame:
    
    list_flops_required = create_list_flops_per_year(df_GPU_park, df_data_GPU, dict_variables, duration_between_demand_multiplications)
    df_month_list = []
    TFlops_required_list = []
    list_demand_peaks = identify_demand_peaks(list_flops_required, dict_variables)
    
    j = 0
    f_bar, c_i = 0, 0
    f_prev = int(df_GPU_park['TFlop_Used'].iloc[-1])
    
    for i in range(nb_months_experiment):
        if j < len(list_demand_peaks) and i == list_demand_peaks[j][2]:
            c_i = list_demand_peaks[j][1]
            f_bar = c_i * tau
            j += 1
        model_res = model(df_data_GPU, df_GPU_park, dict_variables, dict_indicators, list_flops_required[i], lifespan_months, 
                          list_gpu_rate_usage[i], current_year, current_month, estimate_fi_func, f_prev, f_bar, c_i, alpha, beta)
        
        data_month = pd.DataFrame([model_res[1:11]], columns=['Total_GPU_Count', 'Energy_Consumption', 'TFlop_Available', 'TFlop_Used', 'GPU_Added', 
                                                             'GPU_Renewed', 'Price_GPU_Added', 'GPU_Types', 'Indicator_Usage_Totals', 'Usage_Rate'])
        
        f_prev = model_res[11]

        data_month['Month'] = current_month
        data_month['Year'] = current_year
        
        indicator_usage_totals = model_res[9]
        for indicator, usage_total in indicator_usage_totals.items():
            data_month[indicator] = usage_total
        
        TFlops_required_list.append(list_flops_required[i])
        df_month_list.append(data_month)

        df_GPU_park = model_res[0]  # mettre à jour le centre
        
        if current_month<12:
            current_month += 1
        else:
            current_year += 1
            current_month =1
            
        if i == 0: # lors de la création du centre
            data_month['GPU_Added'] = data_month['Total_GPU_Count']
            
    df = pd.concat(df_month_list, ignore_index=True)
    
    df['TFlops_Required'] = TFlops_required_list
    df['Date'] = df['Year'].astype(str) + '-' + df['Month'].astype(str).str.zfill(2)
    df = df.reindex(['Date', 'Total_GPU_Count', 'GPU_Added', 'GPU_Renewed', 'GPU_Types', 'Usage_Rate', 'Energy_Consumption', 'TFlops_Required', 
                     'TFlop_Available', 'TFlop_Used', 'Price_GPU_Added'] + list(indicator_usage_totals.keys()), axis=1)
    
    for indicator in dict_indicators:
        df[f'{indicator}_Production_from_added'] = df.apply(lambda row: resource_consumption_gpu_production(df_data_GPU, row, 'GPU_Added', 
                                                                    dict_variables.get('nb_gpu_regroupement'), dict_indicators[indicator][3], f'{indicator}_Production'), axis=1)
        df[f'{indicator}_Production_from_renewed'] = df.apply(lambda row: resource_consumption_gpu_production(df_data_GPU, row, 'GPU_Renewed', 
                                                                    dict_variables.get('nb_gpu_regroupement'), dict_indicators[indicator][3], f'{indicator}_Production'), axis=1)  

    return df

In [215]:
df_test_model = create_df_test_model(df_data_GPU, df_GPU_park, dict_variables, dict_indicators, dict_variables.get('duration_between_demand_multiplications'),
                                     dict_variables.get('Lifespan_Months'), list_gpu_rate_usage, dict_variables.get('usage_rate'), 
                                     dict_variables.get('nb_months_experiment'), estimate_fi_1, dict_variables.get('current_year'), 
                                     dict_variables.get('current_month'), dict_variables.get('alpha'), dict_variables.get('beta'))

#df_test_model.to_csv('data/df_test_model.csv', index=False)
df_test_model

,Date,Total_GPU_Count,GPU_Added,GPU_Renewed,GPU_Types,Usage_Rate,Energy_Consumption,TFlops_Required,TFlop_Available,TFlop_Used,...,Abiotic_Depletion_Usage,Electronic_Waste_Usage,CO2_Emissions_Production_from_added,CO2_Emissions_Production_from_renewed,Water_Consumption_Production_from_added,Water_Consumption_Production_from_renewed,Abiotic_Depletion_Production_from_added,Abiotic_Depletion_Production_from_renewed,Electronic_Waste_Production_from_added,Electronic_Waste_Production_from_renewed
0,2019-01,8152,8152,0,{'V100': 8152},0.800000,18653080,1019000.0,1019000,8.152000e+05,...,0,0,968050,0,3974100,0,285.32,0.0,23946.5,0.0
1,2019-02,8152,0,0,{'V100': 8152},0.800000,18653080,1019000.0,1019000,8.152000e+05,...,0,0,0,0,0,0,0.00,0.0,0.0,0.0
2,2019-03,8152,0,0,{'V100': 8152},0.800000,18653080,1019000.0,1019000,8.152000e+05,...,0,0,0,0,0,0,0.00,0.0,0.0,0.0
3,2019-04,8152,0,0,{'V100': 8152},0.800000,18653080,1019000.0,1019000,8.152000e+05,...,0,0,0,0,0,0,0.00,0.0,0.0,0.0
4,2019-05,8152,0,0,{'V100': 8152},0.800000,18653080,1019000.0,1019000,8.152000e+05,...,0,0,0,0,0,0,0.00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,2025-08,3192,0,0,{'H100': 3192},0.799998,14658137,6368750.0,6384000,5.107187e+06,...,0,0,0,0,0,0,0.00,0.0,0.0,0.0
80,2025-09,3192,0,0,{'H100': 3192},0.799999,14658154,6368750.0,6384000,5.107193e+06,...,0,0,0,0,0,0,0.00,0.0,0.0,0.0
81,2025-10,3192,0,0,{'H100': 3192},0.799999,14658164,6368750.0,6384000,5.107197e+06,...,0,0,0,0,0,0,0.00,0.0,0.0,0.0
82,2025-11,3192,0,0,{'H100': 3192},0.800000,14658169,6368750.0,6384000,5.107198e+06,...,0,0,0,0,0,0,0.00,0.0,0.0,0.0


___

# Visualisation des données - utilisation de la bibliothèque [**Plotly**](https://plotly.com/python/)

## Divers fonctions : traitement et analyse des Données du Data Center

### Conversion des valeurs numériques

Simplifie les grandes valeurs numériques en unités plus lisibles. Selon que la valeur représente un prix ou non, elle convertit les nombres en préfixes comme Kilo, Mega, Giga, et Téra ou en millions ou milliards. Elle retourne la valeur convertie avec son unité correspondante, arrondie à deux décimales.

In [216]:
def convert_values(value: float, price: bool = False) -> tuple[float, str]:
    value_converted, unit = value, None
    if price==True :
        if value >= 1_000_000_000:
            value_converted = value / 1_000_000_000
            unit = 'Billion' 
        elif value >= 1_000_000:
            value_converted = value / 1_000_000
            unit = 'Million'
        if value_converted > 1:
            unit += 's'
    else:
        if value >= 1_000_000_000_000:
            value_converted = value / 1_000_000_000_000
            unit = 'T'
        elif value >= 1_000_000_000:
            value_converted = value / 1_000_000_000
            unit = 'G'
        elif value >= 1_000_000:
            value_converted = value / 1_000_000
            unit = 'M'
        elif value >= 1_000:
            value_converted = value / 1_000
            unit = 'K'
        
    return round(value_converted, 2), unit or ''

### Identification des années d'ajout de GPU à partir d'une colonne spécifique

In [217]:
def get_list_year_changes(df_test_model: pd.DataFrame, column: str) -> list[pd.Timestamp]:
    list_year_of_gpu_changed = []
    for i in range(df_test_model.shape[0]):
        if df_test_model[column].iloc[i] > 0:
            list_year_of_gpu_changed.append(df_test_model['Date'].iloc[i])
    return list_year_of_gpu_changed

### Fonction pour Générer la liste des années d'ajout de GPU selon le type de raison

La fonction détermine les années spécifiques d'ajout de GPU en fonction du type de raison indiqué. Selon la valeur de `line_indications`, elle calcule les années où des GPU ont été ajoutés pour la demande, le renouvellement, ou les deux. Si l'indication est "Demande", elle soustrait les GPU renouvelés des GPU ajoutés pour obtenir les années pertinentes. Pour "Renewal", elle utilise directement les données des GPU renouvelés, et pour "Demande and Renewal", elle prend en compte tous les GPU ajoutés. Si aucune indication n'est spécifiée, la fonction retourne une liste vide.

In [218]:
def get_line_indications(df_test_model: pd.DataFrame, line_indications: str) -> list[pd.Timestamp]:
    if line_indications == 'None':
        tear_gpu_added = []
    elif line_indications == 'Demande':
        df_test_model['Nb_GPU_added_demand'] = df_test_model['GPU_Added'] - df_test_model['GPU_Renewed']
        tear_gpu_added = get_list_year_changes(df_test_model, 'Nb_GPU_added_demand')
    elif line_indications == 'Renewal':
        tear_gpu_added = get_list_year_changes(df_test_model, 'GPU_Renewed')
    elif line_indications == 'Demande and Renewal':
        tear_gpu_added = get_list_year_changes(df_test_model, 'GPU_Added')
    return tear_gpu_added

### Calcul de la production mensuelle des indicateurs selon la méthode 'Stock'**

Ajoute des colonnes au DataFrame pour la méthode de calcul 'Stock'. Calcule la production mensuelle pour chaque indicateur basé sur les valeurs de production des GPU ajoutés et leur durée de vie. Ces valeurs sont ensuite utilisées pour remplir les colonnes correspondant à la production mensuelle des GPU pendant leur durée de vie, facilitant ainsi la visualisation des émissions de CO2.

In [219]:
def get_df_method_stock(df_test_model: pd.DataFrame, indicator_name: str) -> pd.DataFrame:
    lifespan_months = dict_variables.get('Lifespan_Months')

    df_test_model[f"{indicator_name}_Production_from_added"] = df_test_model[f"{indicator_name}_Production_from_added"].astype(float)
    df_test_model[f"{indicator_name}_per_month_of_life_span"] = 0.0

    for i in range(df_test_model.shape[0]):
        if df_test_model.at[i, f"{indicator_name}_Production_from_added"] > 0:
            production_per_month = df_test_model.at[i, f"{indicator_name}_Production_from_added"] / lifespan_months
            end_index = min(i + lifespan_months, df_test_model.shape[0])
            df_test_model.loc[i:end_index - 1, f"{indicator_name}_per_month_of_life_span"] += production_per_month

    return df_test_model

### Génération des noms des graphiques à partir des indicateurs

La fonction crée une liste de noms de graphiques à partir des clés et valeurs d'un dictionnaire d'indicateurs. Elle reformate les clés du dictionnaire en chaînes lisibles en remplaçant les underscores par des espaces et en ajoutant l'unité de mesure correspondante, tirée des valeurs associées à chaque clé. Cette liste est ensuite utilisée pour générer des titres clairs et informatifs pour les graphiques de visualisation des données.

In [220]:
def get_list_indicators_to_graph(dict_indicators: dict[str, list]) -> list[str]:
    list_graphs_name_indicators = []
    for i in range(len(dict_indicators)):
        name_graph = list(dict_indicators.keys())[i].split('_')
        name_graph = ' '.join(name_graph) + ' in ' +  list(dict_indicators.values())[i][2]
        list_graphs_name_indicators.append(name_graph)
    return list_graphs_name_indicators

In [221]:
list_graphs_name_indicators = get_list_indicators_to_graph(dict_indicators)

___

## Fonctions de visualisation des données basées sur des Widgets interactifs

### TFLops - Scatter plot

Génère un graphique pour visualiser les performances en TFLOPS à l'aide de Plotly. Elle trace trois courbes : les TFLOPS utilisés, les TFLOPS requis et les TFLOPS disponibles. Elle ajoute également des lignes verticales pour indiquer les années de changement significatif de GPU, et ajuste l'axe des ordonnées pour commencer à zéro et s'adapter aux données.

In [222]:
def plot_tflops(fig: go.Figure, df_test_model: pd.DataFrame, row: int, col: int, list_year_of_gpu_changed: list) -> None:
    traces = [
        {'y': df_test_model['TFlop_Used'],
            'name': 'Number of TFLOPS Used',
            'line': {}
        },
        {
            'y': df_test_model['TFlops_Required'],
            'name': 'Number of TFLOPS Required',
            'line': {'dash': 'dash', 'color': 'rgba(255, 0, 0, 0.5)'}
        },
        {
            'y': df_test_model['TFlop_Available'],
            'name': 'Number of TFLOPS Available',
            'line': {'color': 'rgba(0, 255, 0, 0.5)'}
        }
    ]

    for trace in traces:
        fig.add_trace(
            go.Scatter(x=df_test_model['Date'], y=trace['y'], mode='lines', name=trace['name'], line=trace.get('line')),
            row=row, col=col
        )
    
    for i in list_year_of_gpu_changed:
        fig.add_vline(x=i, line_color= 'grey', line_dash = 'dash')
        
    fig.update_yaxes(
        range=[0, None],
        row=row, col=col
    )

### Taux d'usage - Scatter plot 

Trace le taux d'utilisation des GPU sur un graphique. Elle ajoute une courbe représentant le taux d'utilisation des GPU et inclut des lignes verticales pour marquer les années où des changements importants de GPU ont eu lieu.

In [223]:
def plot_gpu_usage_rate(fig: go.Figure, df_test_model: pd.DataFrame, row: int, col: int, list_year_of_gpu_changed: int) -> None:
    fig.add_trace(
        go.Scatter(x=df_test_model['Date'], y=df_test_model['Usage_Rate'], mode='lines', name='GPU Usage Rate', line=dict(color='rgba(0, 255, 255)')),
        row=row, col=col
    )
    for i in list_year_of_gpu_changed:
        fig.add_vline(x=i, line_color= 'grey', line_dash = 'dash')

### Consomation énérgetique - Scatter plot

Génère un graphique montrant la consommation d'énergie du centre de données. Deux options sont proposées : afficher la consommation d'énergie cumulée ou non, selon le paramètre `cumulative`. Le graphique est tracé avec une courbe représentant la consommation d'énergie en kWh, et des lignes verticales sont ajoutées pour indiquer les années de changements importants de GPU. Les axes sont ajustés pour afficher correctement les valeurs de consommation d'énergie.

In [224]:
def plot_data_center_energy_consumption(fig: go.Figure, df_test_model: pd.DataFrame, cumulative: str, row: int, col: int, list_year_of_gpu_changed: list) -> None:
    y = df_test_model['Energy_Consumption'].cumsum() if cumulative == "Cumulative" else df_test_model['Energy_Consumption']
    fig.add_trace(
        go.Scatter(x=df_test_model['Date'], y=y, mode='lines', name='Energy Consumption in kWh', line=dict(color='rgba(255, 255, 0)')),
        row=row, col=col
    )
    for i in list_year_of_gpu_changed:
        fig.add_vline(x=i, line_color='grey', line_dash='dash')
    
    fig.update_yaxes(
        range=[0, None],
        row=row, col=col
    )

#### Nombre de nouveaux GPU ajoutés au Data Center - Bar plot

Crée un graphique à barres illustrant le nombre de GPU ajoutés et renouvelés dans le centre de données. Elle calcule la différence entre les GPU ajoutés pour répondre à la demande et les GPU renouvelés, puis affiche ces valeurs sous forme de barres distinctes sur le graphique. Les lignes verticales marquent les années de changements importants de GPU. Les barres permettent de visualiser les variations mensuelles dans le nombre de GPU ajoutés et renouvelés.

In [225]:
def plot_number_of_gpus_added(fig: go.Figure, df_test_model: pd.DataFrame, row: int, col: int, list_year_of_gpu_changed: list) -> None:
    df_test_model_copy = df_test_model.copy()
    df_test_model_copy['Nb_GPU_added_renewed_diff'] = df_test_model_copy['GPU_Added'] - df_test_model_copy['GPU_Renewed']
    df_test_model_copy.rename(columns={'Nb_GPU_added_renewed_diff': 'Number of GPUs added to meet demand',
                                       'GPU_Renewed': 'Number of GPUs renewed'}, inplace=True)
    
    fig.add_trace(
        go.Bar(x=df_test_model['Date'], y=df_test_model_copy['Number of GPUs added to meet demand'], name='Number of GPUs added to meet demand'),
        row=row, col=col
    )
    fig.add_trace(
        go.Bar(x=df_test_model['Date'], y=df_test_model_copy['Number of GPUs renewed'], name='Number of GPUs renewed'),
        row=row, col=col
    )
    for i in list_year_of_gpu_changed:
        fig.add_vline(x=i, line_color= 'grey', line_dash = 'dash')

### Coût des GPU - Scatter plot

Trace le coût des GPU ajoutés au centre de données sur un graphique, permettant de suivre l'évolution du coût des GPU ajoutés au fil du temps.. Ainis offre la possibilité de visualiser le coût total sous forme cumulative ou non, en fonction de l'argument `cumulative`. Les lignes verticales marquent les années où des changements significatifs ont eu lieu dans l'ajout de GPU. 

In [226]:
def plot_data_center_price(fig: go.Figure, df_test_model: pd.DataFrame, cumulative: str, row: int, col: int, list_year_of_gpu_changed: list) -> None:
    y = df_test_model['Price_GPU_Added'].cumsum() if cumulative == "Cumulative" else df_test_model['Price_GPU_Added']
    fig.add_trace(
        go.Scatter(x=df_test_model['Date'], y=y, mode='lines', name='Coût des GPU ajoutés', line=dict(color='rgba(255, 255, 0)')),
        row=row, col=col
    )
    for i in list_year_of_gpu_changed:
        fig.add_vline(x=i, line_color='grey', line_dash='dash')
    fig.update_yaxes(
        range=[0, None],
        row=row, col=col
    )

### Types des GPU - Bar plot

Génère un graphique à barres empilées montrant la répartition des types de GPU au fil du temps. Elle convertit les dates, normalise les types de GPU en colonnes, et utilise des couleurs spécifiques pour chaque type. Les barres empilées illustrent la distribution cumulée des GPU, avec des lignes verticales marquant les années de changement.

In [227]:
def plot_gpu_distribution(fig: go.Figure, df: pd.DataFrame, row: int, col: int, list_year_of_gpu_changed: list) -> None:
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m')
    df_gpu_types = pd.json_normalize(df['GPU_Types']).fillna(0)
    dates = df['Date'].dt.strftime('%Y-%m')
    
    default_colors = {'V100': 'red', 'A100': 'orange', 'H100': 'green'}
    colors = {gpu_type: default_colors.get(gpu_type, 'grey') for gpu_type in df_gpu_types.columns}

    bottoms = pd.Series([0] * len(df))
    
    for gpu_type in df_gpu_types.columns:
        fig.add_trace(go.Bar(
            x=dates,
            y=df_gpu_types[gpu_type],
            name=gpu_type,
            marker_color=colors[gpu_type],
            base=bottoms,
            showlegend=(row == 1 and col == 1)
        ), row=row, col=col)
        bottoms += df_gpu_types[gpu_type]
    
    for i in list_year_of_gpu_changed:
        fig.add_vline(x=i, line_color= 'grey', line_dash = 'dash')

    fig.update_layout(
        barmode='stack',
    )

###  Indicateurs du dictionnaire `dict_indicators` - Scatter plot

La fonction `add_traces` ajoute des courbes ou des histogrammes, selon le type de graphique, pour visualiser l'utilisation et la production de données. `get_ecumulative_data` prépare les données en cumulant les valeurs si nécessaire et retourne les séries pour l'utilisation et la production. Enfin, `plot_indicators` crée un graphique pour les indicateurs choisis, en utilisant `add_traces` pour afficher les données et marquer les changements de GPU avec des lignes verticales.

In [228]:
def add_traces(fig: go.Figure, df: pd.DataFrame, y_1: pd.Series, y_2: pd.Series, type_graph: str, row: int, col: int, name_graph: str, unit: str) -> None:
    if type_graph == 'Scatter':
        fig.add_trace(
            go.Scatter(x=df['Date'], y=y_2, name=f'{name_graph} production in {unit}', stackgroup='one', line=dict(color="#EB6180")),
            row=row, col=col
        )
        fig.add_trace(
            go.Scatter(x=df['Date'], y=y_1, name=f'{name_graph} usage in {unit}', stackgroup='one', line=dict(color="#78EAAC")),
            row=row, col=col
        )
    else:
        fig.add_trace(
            go.Bar(x=df['Date'], y=y_1, name=f'{name_graph} usage in {unit}', marker=dict(color="#78EAAC")),
            row=row, col=col
        )
        fig.add_trace(
            go.Bar(x=df['Date'], y=y_2, name=f'{name_graph} production in {unit}', marker=dict(color="#EB6180")),
            row=row, col=col
        )
        fig.update_layout(barmode='stack')

def get_ecumulative_data(df: pd.DataFrame, cumulative: str, usage_col: str, production_col: str) -> tuple[pd.Series, pd.Series]:
    df[usage_col] = df[usage_col].astype(float)
    df[production_col] = df[production_col].astype(float)
    
    y_1 = df[usage_col].cumsum() if cumulative == "Cumulative" else df[usage_col]
    y_2 = df[production_col].cumsum() if cumulative == "Cumulative" else df[production_col]
    return y_1, y_2

def plot_indicators(fig: go.Figure, df_test_model: pd.DataFrame, cumulative: str, type_graph: str, methode_production_indicators: str, row: int, col: int, list_year_of_gpu_changed: list[str], indicator_name: str, unit: str) -> None:
    if methode_production_indicators == 'Flux':
        df_test_model[f'{indicator_name}_Total'] = df_test_model[f'{indicator_name}_Usage'] + df_test_model[f'{indicator_name}_Production_from_added']
        y_1, y_2 = get_ecumulative_data(df_test_model, cumulative, f'{indicator_name}_Usage', f'{indicator_name}_Production_from_added')
    else:
        df_test_model = get_df_method_stock(df_test_model, indicator_name)
        df_test_model[f'{indicator_name}_Total'] = df_test_model[f'{indicator_name}_Usage'] + df_test_model[f"{indicator_name}_per_month_of_life_span"]
        y_1, y_2 = get_ecumulative_data(df_test_model, cumulative, f'{indicator_name}_Usage', f"{indicator_name}_per_month_of_life_span")

    name_graph = indicator_name.split('_')
    name_graph = ' '.join(name_graph)
    add_traces(fig, df_test_model, y_1, y_2, type_graph, row, col, name_graph, unit)
    
    for i in list_year_of_gpu_changed:
        fig.add_vline(x=i, line_color='grey', line_dash='dash')

___

## Initiation des [Widgets interactifs](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html) pour la visualisation dynamique des données

Les widgets permettent une personnalisation et une visualisation dynamique des données sur les GPU, incluant le choix des graphiques, la sélection de la fonction d'estimation, et l'ajustement des paramètres du modèle. Ils offrent aussi des options pour afficher des données spécifiques comme la consommation d'énergie, les coûts des GPU et divers autres indicateurs. 

Voici la création de ces widgets :

In [229]:
graphs_to_display_widget = widgets.TagsInput(
    value=['Number of TFLOPS', 'GPU Usage Rate', 'Energy Consumption in kWh', 'Number of GPUs Added', 'Price of the GPU', 'Type of GPU'] + list_graphs_name_indicators,
    allowed_tags=['Number of TFLOPS', 'GPU Usage Rate', 'Energy Consumption in kWh', 'Number of GPUs Added', 'Price of the GPU', 'Type of GPU'] + list_graphs_name_indicators,
    allow_duplicates=False
)

estimate_fi_func_widget = widgets.Dropdown(
    options={'estimate_fi_1': estimate_fi_1, 'estimate_fi_2': estimate_fi_2, 'estimate_fi_3': estimate_fi_3}, description='Estimate Function',
    layout=Layout(width='60%', height='25px'), style={'description_width': '220px'}, continuous_update=False
)

duration_widget = widgets.IntSlider(
    value=dict_variables.get('duration_between_demand_multiplications'), min=3, max=dict_variables.get('nb_months_experiment'), step=1, description='Duration of Demand Multiplications',
    layout=Layout(width='60%', height='25px'), style={'description_width': '220px'}, continuous_update=False
)

lifespan_months_widget = widgets.IntSlider(
    value=dict_variables.get('Lifespan_Months'), min=1, max=dict_variables.get('nb_months_experiment'), step=1, description='Lifespan of a GPU (months)', 
    layout=Layout(width='60%', height='25px'), style={'description_width': '220px'}, continuous_update=False
)

line_indications_widget = widgets.RadioButtons(
    options=['None', 'Demande', 'Renewal', 'Demande and Renewal'], description='Line indicating the new GPU',
    layout=Layout(width='60%', height='120px'), style={'description_width': '220px'}, disabled=False
)

alpha_widget = widgets.FloatSlider(
    value=dict_variables.get('alpha'), min=0.01, max=1, step=0.01, description='Alpha',
    layout=Layout(width='45%', height='120px'), style={'description_width': '220px'}, continuous_update=False
)

beta_1_widget = widgets.FloatSlider(
    value=dict_variables.get('beta'), min=0.01, max=1.5, step=0.01, description='Beta 1',
    layout=Layout(width='45%', height='120px'), style={'description_width': '220px'}, continuous_update=False
)

beta_2_widget = widgets.FloatSlider(
    value=dict_variables.get('beta'), min=0.01, max=1, step=0.01, description='Beta 2',
    layout=Layout(width='45%', height='120px'), style={'description_width': '220px'}, continuous_update=False
)

methode_production_indicators_widget = widgets.RadioButtons(
    options=['Flux', 'Stock'], description='Type of methode',
    layout=Layout(width='60%', height='70px'), style={'description_width': '220px'}, disabled=False
)

type_graph_cumulative_widget = widgets.RadioButtons(
    options=['Differential', 'Cumulative'], description='Data Types for Electricity and CO2',
    layout=Layout(width='60%', height='70px'), style={'description_width': '220px'}, disabled=False
)

graph_type_widget = widgets.RadioButtons(
    options=['Scatter', 'Bar'],
    description='Graph Type',
    layout=Layout(width='60%', height='70px'),
    style={'description_width': '220px'},
    disabled=False
)

energy_consumption_widget = widgets.HTML()
gpus_added_widget = widgets.HTML()
price_gpu_widget = widgets.HTML()
co2_emission_usage_widget = widgets.HTML()
co2_emissions_gpu_prod_widget = widgets.HTML()
water_used_gpu_usage_widget = widgets.HTML()
water_used_gpu_prod_widget = widgets.HTML()
abiotic_depletion_gpu_usage_widget = widgets.HTML()
abiotic_depletion_gpu_prod_widget = widgets.HTML()
electronic_waste_gpu_prod_widget = widgets.HTML()

Pour chaque clé dans `dict_indicators`, un widget de type bouton radio est généré, permettant de choisir entre les graphiques de type "Scatter" ou "Bar". Ces widgets sont ensuite stockés dans une liste, avec leurs noms dynamiques générés et enregistrés dans l'espace global pour une utilisation ultérieure. Les widgets et les indicateurs associés sont organisés en trois listes distinctes : `list_widgets_indicators` pour les widgets, `list_indicators` pour les noms des indicateurs, et `list_widget_name` pour les noms des widgets.

In [230]:
i = 0
list_widget_name = []
list_widgets_indicators = []
list_indicators = []

for key in dict_indicators.keys():
    widget_name = f'{key}_type_widget'
    widget = widgets.RadioButtons(
        options=['Scatter', 'Bar'], description=f'Type of the {list_graphs_name_indicators[i]} Graph',
        layout=Layout(width='60%', height='70px'), style={'description_width': '220px'}, disabled=False
    )
    list_widgets_indicators.append(widget)
    indicators = f'{key.lower()}'
    list_indicators.append(indicators)
    list_widget_name.append(widget_name)
    globals()[widget_name] = widget
    i += 1    

___

## Graphiques

La fonction `update_alpha_beta_visibility` ajuste la visibilité des widgets pour les paramètres alpha et beta en fonction de la fonction d'estimation sélectionnée. La fonction `update_visibility` rend visibles les widgets relatifs aux types de graphes et aux affichages spécifiques selon les graphiques sélectionnés. La fonction `update_plot` génère les graphiques en fonction des sélections et des paramètres ajustés, calcule divers indicateurs, et met à jour les widgets d'affichage avec les résultats. Ces widgets affichent des informations détaillées telles que la consommation d'énergie, le coût des GPU, et les émissions de CO2.

In [231]:
def update_alpha_beta_visibility(change: dict[str, any]) -> None:
    fonction_to_dispay = change['new']
    if fonction_to_dispay == estimate_fi_1:
        alpha_widget.layout.display = 'block'
        beta_1_widget.layout.display = 'none'
        beta_2_widget.layout.display = 'none'
    elif fonction_to_dispay == estimate_fi_2:
        alpha_widget.layout.display = 'none'
        beta_1_widget.layout.display = 'block'
        beta_2_widget.layout.display = 'none'
    elif fonction_to_dispay == estimate_fi_3:
        alpha_widget.layout.display = 'none'
        beta_1_widget.layout.display = 'none'
        beta_2_widget.layout.display = 'block'

estimate_fi_func_widget.observe(update_alpha_beta_visibility, names='value')
update_alpha_beta_visibility({'new': estimate_fi_func_widget.value})

def update_visibility(change: dict[str, any]) -> None:
    graphs_to_display = change['new']
    
    if any(graph in graphs_to_display for graph in ['Energy Consumption in kWh'] + list_graphs_name_indicators):
        type_graph_cumulative_widget.layout.display = 'block'
    
graph_type_widget.observe(update_visibility, names='value')
update_visibility({'new': graphs_to_display_widget.value})

graphs_to_display_widget.observe(update_visibility, names='value')
update_visibility({'new': graphs_to_display_widget.value})

def update_plot(graphs_to_display: list[str], duration_between_demand_multiplications: int, lifespan_months: int, estimate_fi_func: Callable, alpha :float, beta_1: float, beta_2: float, line_indications: str, type_graph_cumulative: str, methode_production_indicators: str, graph_type: str):
    pio.renderers.default = 'notebook'
    df_GPU_park = df_GPU_park_init.copy()
    beta_value = beta_1 if estimate_fi_func == estimate_fi_2 else beta_2 if estimate_fi_func == estimate_fi_3 else dict_variables.get('beta')
    
    test_model = create_df_test_model(df_data_GPU, df_GPU_park, dict_variables, dict_indicators, duration_between_demand_multiplications, lifespan_months, list_gpu_rate_usage, 
                                      dict_variables.get('usage_rate'), dict_variables.get('nb_months_experiment'), estimate_fi_func,
                                      dict_variables.get('current_year'), dict_variables.get('current_month'),
                                      alpha=alpha if estimate_fi_func == estimate_fi_1 else dict_variables.get('alpha'), beta=beta_value)
    
    df_test_model = test_model
    list_year_of_gpu_changed = get_line_indications(df_test_model, line_indications)
    
    num_graphs = len(graphs_to_display)
    num_rows = math.ceil(num_graphs / 2)
    num_cols = 2 if num_graphs > 1 else 1
    
    fig = make_subplots(rows=num_rows, cols=num_cols, subplot_titles=tuple(graphs_to_display))
    row, col = 1, 1

    if 'Number of TFLOPS' in graphs_to_display:
        plot_tflops(fig, df_test_model, row, col, list_year_of_gpu_changed)
        col += 1
        if col > num_cols:
            col = 1
            row += 1
    
    if 'GPU Usage Rate' in graphs_to_display:
        plot_gpu_usage_rate(fig, df_test_model, row, col, list_year_of_gpu_changed)
        col += 1
        if col > num_cols:
            col = 1
            row += 1
    
    if 'Energy Consumption in kWh' in graphs_to_display:
        plot_data_center_energy_consumption(fig, df_test_model, type_graph_cumulative, row, col, list_year_of_gpu_changed)
        col += 1
        if col > num_cols:
            col = 1
            row += 1
    
    if 'Number of GPUs Added' in graphs_to_display:
        plot_number_of_gpus_added(fig, df_test_model, row, col, list_year_of_gpu_changed)
        col += 1
        if col > num_cols:
            col = 1
            row += 1
    
    if 'Price of the GPU' in graphs_to_display:
        plot_data_center_price(fig, df_test_model, type_graph_cumulative, row, col, list_year_of_gpu_changed)
        col += 1
        if col > num_cols:
            col = 1
            row += 1
            
    if 'Type of GPU' in graphs_to_display:
        plot_gpu_distribution(fig, df_test_model, row, col, list_year_of_gpu_changed)
        col += 1
        if col > num_cols:
            col = 1
            row += 1

    for i, indicator in enumerate(list_indicators):
        indicator_type = list_graphs_name_indicators[i]
        if indicator_type in graphs_to_display:
            plot_indicators(fig, df_test_model, type_graph_cumulative, graph_type, methode_production_indicators, row, col, list_year_of_gpu_changed, list(dict_indicators.keys())[i], list(dict_indicators.values())[i][2])
            col += 1
            if col > num_cols:
                col = 1
                row += 1
    
    fig.update_layout(title_text='Analysis')
    energy_consumption = convert_values(calculate_total_sum_variable(df_test_model, 'Energy_Consumption', float)) * 1000
    gpus_added = calculate_total_sum_variable(df_test_model, 'GPU_Added', int)
    price_gpu = convert_values(calculate_total_sum_variable(df_test_model, 'Price_GPU_Added', int), True)
    co2_emission_usage = convert_values(calculate_total_sum_variable(df_test_model, 'CO2_Emissions_Usage', float))
    co2_emissions_gpu_prod = convert_values(calculate_total_sum_variable(df_test_model, 'CO2_Emissions_Production_from_added', float), False)
    water_used_gpu_usage = convert_values(calculate_total_sum_variable(df_test_model, 'Water_Consumption_Usage', float), False)
    water_used_gpu_prod = convert_values(calculate_total_sum_variable(df_test_model, 'Water_Consumption_Production_from_added', float), False)
    abiotic_depletion_gpu_usage = convert_values(calculate_total_sum_variable(df_test_model, 'Abiotic_Depletion_Usage', float), False)
    abiotic_depletion_gpu_prod = convert_values(calculate_total_sum_variable(df_test_model, 'Abiotic_Depletion_Production_from_added', float), False)
    electronic_waste_gpu_prod = convert_values(calculate_total_sum_variable(df_test_model, 'Electronic_Waste_Production_from_added', float), False)
    
    energy_consumption_widget.value = f"<b>Energy Consumption: </b> {energy_consumption[0]} {energy_consumption[1]} kWh"
    gpus_added_widget.value = f"<b>GPUs Added:</b> {gpus_added} units"
    price_gpu_widget.value = f"<b>Price of GPUs Added:</b> {price_gpu[0]} {price_gpu[1]} euros"
    co2_emission_usage_widget.value = f"<b>CO2 Emissions Usage:</b> {co2_emission_usage[0]} {co2_emission_usage[1]}kg"
    co2_emissions_gpu_prod_widget.value = f"<b>CO2 Emissions from GPU Production:</b> {co2_emissions_gpu_prod[0]} {co2_emissions_gpu_prod[1]}kg"
    water_used_gpu_usage_widget.value = f"<b>Water Usage:</b> {water_used_gpu_usage[0]} {water_used_gpu_usage[1]}L"
    water_used_gpu_prod_widget.value = f"<b>Water from GPU Production:</b> {water_used_gpu_prod[0]} {water_used_gpu_prod[1]}L"
    abiotic_depletion_gpu_usage_widget.value = f"<b>Abiotic Depletion Usage:</b> {abiotic_depletion_gpu_usage[0]} {abiotic_depletion_gpu_usage[1]}kg/Sbeq"
    abiotic_depletion_gpu_prod_widget.value = f"<b>Abiotic Depletion from GPU Production:</b> {abiotic_depletion_gpu_prod[0]} {abiotic_depletion_gpu_prod[1]}kg/Sbeq"
    electronic_waste_gpu_prod_widget.value = f"<b>Electronic Waste from GPU Production:</b> {electronic_waste_gpu_prod[0]} {electronic_waste_gpu_prod[1]}kg"
    
    fig.show()
    display(energy_consumption_widget, gpus_added_widget, price_gpu_widget, co2_emission_usage_widget, co2_emissions_gpu_prod_widget, water_used_gpu_usage_widget, water_used_gpu_prod_widget,
            abiotic_depletion_gpu_usage_widget, abiotic_depletion_gpu_prod_widget, electronic_waste_gpu_prod_widget)

Les graphiques suivants permettent de visualiser l'évolution de :

- **Nombre de TFLOPS** : indique la puissance de calcul totale disponible
- **Taux d'utilisation des GPU** : montre à quel point les GPU sont utilisés par rapport à leur capacité totale (mesurée entre 0 et 1)
- **Consommation du datacenter en kWh** : représente la consommation énergétique du datacenter en kWh
- **Nombre de GPU ajoutés** : montre le nombre de nouveaux GPU ajoutés au parc pour satisfaire la demande
- **Coût des GPU** : affiche les dépenses associées à l'ajout de nouveaux GPU en €
- **Type de GPU dans le datacenter** : illustre la répartition des différents types de GPU présents dans le datacenter

Les widgets suivants permettent de modifier les valeurs des différentes variables :

- Un champ permettant de sélectionner les graphiques à afficher parmi une liste prédéfinie, comme "Nombre de TFLOPS", "Taux d'utilisation des GPU", "Consommation du datacenter en kWh", etc
- Le menu déroulant **"Estimate Function"** permet de choisir parmi trois fonctions d'estimation différentes pour l'approximation de la capacité maximale
- Le curseur **"Duration between demand multiplications"** ajuste le nombre de mois après lesquels la demande de TFLOPS sera multipliée par 2,5, avec une valeur de base définie dans le dictionnaire `dict_variables`
- Le curseur **"Lifespan of a GPU"** modifie la durée de vie maximale d'un GPU, indiquant le moment où les GPU doivent être remplacés.
- Les curseurs **"Alpha"** et **"Beta"** ajustent les paramètres d'estimation de la capacité maximale utilisée au fil du temps :
  - Pour la fonction `estimate_fi_1`, l'alpha varie entre 0.01 et 1 avec un pas de 0.01
  - Pour la fonction `estimate_fi_2`, le beta varie entre 0.01 et 1,5 avec un pas de 0.01 (au-delà de 1.5, les valeurs restent constantes)
  - Pour la fonction `estimate_fi_3`, le beta varie entre 0.01 et 1 avec un pas de 0.01
- **"Line indications"** : un groupe de boutons radio permettant de choisir les lignes à afficher sur les graphiques, telles que la demande ou le renouvellement
- **"Data Types for Electricity and CO2"** : des boutons radio pour sélectionner le type de données à afficher, soit "Differential" soit "Cumulative"
- **"Type of method"** : des boutons radio pour choisir entre les méthodes de production "Flux" ou "Stock"
- **"Graph Type"** : un groupe de boutons radio pour sélectionner le type de graphique à afficher, soit "Scatter" soit "Bar"

À la fin des graphiques, une analyse globale est réalisée pour évaluer l'impact environnemental sur l'ensemble de la période de l'expérience.

In [232]:
interact(
    update_plot,
    graphs_to_display=graphs_to_display_widget,
    duration_between_demand_multiplications=duration_widget,
    lifespan_months=lifespan_months_widget,
    estimate_fi_func=estimate_fi_func_widget,
    alpha=alpha_widget,
    beta_1=beta_1_widget,
    beta_2=beta_2_widget,
    line_indications=line_indications_widget,
    type_graph_cumulative=type_graph_cumulative_widget,
    methode_production_indicators=methode_production_indicators_widget,
    graph_type=graph_type_widget
);

interactive(children=(TagsInput(value=['Number of TFLOPS', 'GPU Usage Rate', 'Energy Consumption in kWh', 'Num…

___

# Fonction de calcul de la surface en m² utilisée par le datacenter

Dans cette section, nous allons développer une fonction permettant de calculer la surface totale en mètres carrés utilisée par le datacenter.

### Dictionnaire des variables techniques pour l'aménagement de la surface du Data Cente

#### Quelques informations clés :

Dans une salle de serveurs typique, les racks sont souvent disposés en rangées avec des couloirs froids situés entre ces rangées de racks. Pour une rangée de racks : La Largeur du couloir froid est comptée à l'avant de la rangée de racks, ainsi elle est également comptée à l'arrière de la rangée de racks. De plus, un couloir froid supplémentaire est nécessaire entre chaque paire de lignes de racks pour assurer une bonne circulation de l'air.

#### Données techniques : 
 - Taille des Serveurs :
  
   - Unité de Rack (U) : les racks de serveurs sont mesurés en "unités de rack" notés "U". Nous allons nous concentrer sur les serveurs du type "Serveur 4U" qui permetent d'accueillir jusqu'à 8 GPU ou plus (en fonction du modèle)
  
- Taille des Racks :
  - [Racks de 42U](https://www.42u.com/42U-cabinets.htm) : un rack de serveur industriel standard mesure généralement 42U de hauteur, ce qui correspond à environ 2 mètres
    - largeur : **0.6 m ou 0.8 m**
    - profondeur : **0.8 m ou 1,05 m**

- La salle se compose de [**deux types de couloir**](https://www.techtarget.com/searchdatacenter/definition/hot-cold-aisle), qui doivent être aménagées autour de chaque rack :
  - Couloir froide (Cold Aisle) :
    - l'air frais est aspiré par les serveurs
    - généralement une largeur de [**1,2 mètres**](https://lacltd.uk.com/data-centre-hot-aisle-cold-aisle-)
    - permet un bon flux d'air et un accès suffisant pour le personnel

  - Couloir chaude (Hot Aisle) :
    - l'air chaud est expulsé à l'arrière des serveurs 
    - largeur recommandée de [**1,2 mètres**](https://lacltd.uk.com/data-centre-hot-aisle-hot-aisle-) 
    - elle peut parfois être plus étroite, en fonction de l'espace disponible et du design de la salle

- Espacement entre les Racks et le mur : [**1.2 m**](https://www.smartisystems.com/data-center/a-definitive-guide-of-standard-server-room/#:~:text=The%20arrangement%20of%20equipment%20should,at%20the%20rear%20is%20recommended)

Ce dictionnaire contient des variables clés pour la configuration physique du Data Center, toutes les valeurs étant exprimées en mètres.

- **`high_42U_server_racks`** : nombre total de racks de serveurs de 42U dans l'installation (type : int)

- **`width_42U_server_racks`** : largeur d'un rack de serveur de 42U (type : float)

- **`depth_42U_server_racks`** : profondeur d'un rack de serveur de 42U (type : float)

- **`width_hot_aisle`** : largeur d'une allée chaude dans le centre de données (type : float)

- **`width_cold_aisle`** : largeur d'une allée froide dans le centre de données (type : float)

- **`rack_42U`** : hauteur d'un rack de serveur en unités de rack (U), où une unité correspond à environ 4,45 cm (type : int)

- **`4U_server`** : taille d'un serveur en unités de rack (U), spécifiant qu'il occupe 4U de hauteur dans un rack (type : int)

- **`gpu_per_server`** : nombre de GPU installés dans un serveur (type : int)

- **`num_of_racks_in_a_line`** : nombre de racks pouvant être installés en ligne dans une disposition donnée (type : int)

- **` num_of_racks_in_a_line_allowed`** : nombre maximal de racks autorisés en ligne, utilisé principalement pour déterminer la capacité maximale d'une ligne de racks dans l'outil interactif. Cette variable sert à définir les limites supérieures pour la configuration visuelle et ne reflète pas nécessairement les contraintes physiques ou opérationnelles réelles (type : int)

In [233]:
dict_variables_servers = {"high_42U_server_racks": 2,
                    "width_42U_server_racks": 0.8, 
                    "depth_42U_server_racks": 1.05,
                    "width_hot_aisle": 1.5, 
                    "width_cold_aisle": 1.5, 
                    "rack_42U": 42,
                    "4U_server": 4,
                    "gpu_per_server": 8,
                    "num_of_racks_in_a_line": 20,
                    "num_of_racks_in_a_line_allowed": 150,
                    }

### Calcul de la surface nécessaire par les racks de serveurs dans un Datacenter

La fonction calcule la surface totale occupée par les racks de serveurs dans un datacenter en fonction du nombre de GPU et des dimensions des racks. Détermine également le nombre de racks, de lignes, ainsi que le nombre de couloirs froids et chauds nécessaires pour assurer une disposition efficace et optimale des serveurs, retournés dans un dictionnaire.

Fonction pour calculer la surface utilisée :

$$
\text{Surface par rack en m}^2 = \frac{(\text{Profondeur du rack} + \text{Nombre de couloirs chaud/froid} \times \text{Largeur d'un couloir}) \times (\text{Longueur du rack} \times \text{Nombre de racks} + 2 \times \text{Largeur du couloir froid})}{\text{Nombre de racks dans la salle}}
$$

In [234]:
def size_server_room(dict_variables_servers: dict[str, int], num_gpu: int,  num_of_racks_in_a_line: int = dict_variables_servers.get('num_of_racks_in_a_line')) -> dict[str, float]:
    rack_42U = dict_variables_servers.get('rack_42U')
    U_server = dict_variables_servers.get('4U_server')
    gpu_per_server = dict_variables_servers.get('gpu_per_server')    
    width_42U_server_racks = dict_variables_servers.get('width_42U_server_racks')
    depth_42U_server_racks = dict_variables_servers.get('depth_42U_server_racks')
    width_cold_aisle = dict_variables_servers.get('width_cold_aisle')
    width_hot_aisle = dict_variables_servers.get('width_hot_aisle')

    num_servers_rack = rack_42U // U_server  # nombre de serveurs par rack
    num_servers_needed = math.ceil(num_gpu / gpu_per_server)  # nombre total de serveurs nécessaires
    num_of_racks_needed = math.ceil(num_servers_needed / num_servers_rack)  # nombre de racks nécessaires
    num_of_lines = math.ceil(num_of_racks_needed /  num_of_racks_in_a_line)  # nombre de lignes de racks nécessaires
    num_of_lines = num_of_lines + 1 if num_of_lines % 2 == 1 else num_of_lines
     
    total_width = (width_42U_server_racks *  num_of_racks_in_a_line) + 2 * width_cold_aisle

    # nombre de couloirs froids et chauds nécessaires
    num_cold_aisles = math.ceil((num_of_lines + 1) / 2)  # un couloir froid entre chaque paire de lignes
    num_hot_aisles = num_cold_aisles - 1

    total_depth = (depth_42U_server_racks * num_of_lines) + (num_cold_aisles * width_cold_aisle) + (num_hot_aisles * width_hot_aisle)

    return {
        'total_width': total_width,
        'total_depth': total_depth,
        'num_of_racks_needed': num_of_racks_needed,
        'num_cold_aisles': num_cold_aisles,
        'num_hot_aisles': num_hot_aisles,
        'num_of_lines': num_of_lines
    }

### Calcul de la surface totale et de l'espace occupé par les racks de serveurs dans un Datacenter

Donne la surface totale utilisée par les racks de serveurs dans un datacenter, en prenant en compte les dimensions du datacenter, le nombre de racks nécessaires, et la disposition des lignes de racks. La fonction retourne trois valeurs : la surface totale, la surface réellement utilisée par les racks, et le pourcentage de la surface totale effectivement occupée par les racks.

In [235]:
def area_used_servers(dimensions: dict[str, int], num_racks_per_line) -> tuple[float, float, float]:
    total_width = dimensions['total_width']
    total_depth = dimensions['total_depth']
    num_of_racks_needed = dimensions['num_of_racks_needed']
    num_of_lines = dimensions['num_of_lines']
    
    total_area = (total_width * total_depth) 
    used_area = total_area / (num_of_lines * num_racks_per_line) * num_of_racks_needed
    percentage_used_area =  used_area / total_area * 100
    return round(total_area, 2), round(used_area, 2), round(percentage_used_area, 2)

### Graphiques en utilisant [matplotlib.patches.Rectangle](https://matplotlib.org/stable/api/_as_gen/matplotlib.patches.Rectangle.html)

Le code convertit la colonne "Date" de `df_test_model` en un format datetime, puis extrait les dates uniques au format "YYYY-MM" pour les utiliser comme options.

In [236]:
df_test_model['Date'] = pd.to_datetime(df_test_model['Date'], format='%Y-%m')

date_options = df_test_model['Date'].dt.strftime('%Y-%m').unique()

Création des Widgets :

In [237]:
date_slider = widgets.SelectionSlider(
    options=date_options,
    value=date_options[0],
    description='Date',
    continuous_update=False,
    orientation='horizontal',
    layout=Layout(width='60%', height='25px'),
    style={'description_width': '100px'}, 
)

num_racks_slider = widgets.IntSlider(
    value=dict_variables_servers.get('num_of_racks_in_a_line'), 
    min=1, 
    max=dict_variables_servers.get('num_of_racks_in_a_line_allowed'), 
    step=1, 
    description='Racks per Line',
    continuous_update=False,
    orientation='horizontal',
    layout=Layout(width='60%', height='25px'),
    style={'description_width': '100px'},
)

info_widget = widgets.HTML(value="")

### Visualisation de la disposition des racks dans le Datacenter - Scatter plot

Génère une visualisation de la disposition des racks dans un datacenter pour une date spécifique. Dessine les racks en tenant compte de la largeur et de la profondeur totales du datacenter, ainsi que des dimensions des racks et des allées. Les racks sont colorés pour indiquer leur disponibilité, et les allées chaudes et froides sont marquées pour montrer leur placement.

In [238]:
def plot_server_layout(ax: plt.Axes, selected_date: datetime.date, dict_variables_servers: dict[str, int], num_racks: int, dimensions: dict[str, int]) ->  None:
    total_width = dimensions['total_width']
    total_depth = dimensions['total_depth']
    num_of_racks_needed = dimensions['num_of_racks_needed']
    num_cold_aisles = dimensions['num_cold_aisles']
    num_hot_aisles = dimensions['num_hot_aisles']
    num_of_lines = dimensions['num_of_lines']
    
    width_cold_aisle = dict_variables_servers.get('width_cold_aisle')
    width_hot_aisle = dict_variables_servers.get('width_hot_aisle')
    rack_width = dict_variables_servers.get('width_42U_server_racks')
    rack_depth = dict_variables_servers.get('depth_42U_server_racks')
    
    current_depth = 0
    cold_aisle_x = width_cold_aisle
    cold_aisle_y = current_depth
    rect = plt.Rectangle((cold_aisle_x, cold_aisle_y), total_width - 2 * width_cold_aisle, width_cold_aisle, facecolor='lightblue')
    ax.add_patch(rect)
    current_depth += width_cold_aisle
    racks_to_count = 0

    for line in range(num_of_lines):
        for rack in range(num_racks):
            rack_x = cold_aisle_x + rack * rack_width
            rack_y = current_depth
            color = 'lightgreen' if racks_to_count < num_of_racks_needed else 'lightgray'
            edge_color = 'green' if racks_to_count < num_of_racks_needed else 'gray'
            ax.add_patch(plt.Rectangle((rack_x, rack_y), rack_width, rack_depth, edgecolor=edge_color, facecolor=color))
            racks_to_count += 1
        
        current_depth += rack_depth

        if line % 2 == 0 and num_hot_aisles > 0:
            ax.add_patch(plt.Rectangle((cold_aisle_x, current_depth), total_width - 2 * width_cold_aisle, width_hot_aisle, edgecolor='red', facecolor='lightcoral'))
            current_depth += width_hot_aisle
            num_hot_aisles -= 1
        elif line % 2 == 1 and num_cold_aisles > 0:
            ax.add_patch(plt.Rectangle((cold_aisle_x, current_depth), total_width - 2 * width_cold_aisle, width_cold_aisle, facecolor='lightblue'))
            current_depth += width_cold_aisle
            num_cold_aisles -= 1
    
    rect_1 = plt.Rectangle((0, 0), width_cold_aisle, total_depth, facecolor='lightblue')  
    rect_2 = plt.Rectangle((total_width - width_cold_aisle, 0), width_cold_aisle, total_depth, facecolor='lightblue')
    ax.add_patch(rect_1)
    ax.add_patch(rect_2)
    
    ax.set_xlim(0, total_width)
    ax.set_ylim(0, total_depth)
    ax.set_aspect('equal')
    ax.invert_yaxis()
    
    ax.set_xlabel('Width in meters')
    ax.set_ylabel('Depth in meters')
    ax.set_title(f'Server room organisation for {selected_date.strftime("%Y-%m")}')
    
    hot_aisle_patch = patches.Patch(color='lightcoral', label='Hot aisle')
    cold_aisle_patch = patches.Patch(color='lightblue', label='Cold aisle')
    rack_patch = patches.Rectangle((0, 0), 1, 1, edgecolor='gray', facecolor='lightgray', label='Server rack availble')
    rack_patch_used = patches.Rectangle((0, 0), 1, 1, edgecolor='green', facecolor='lightgreen', label='Server rack used or partially used')
    ax.legend(handles=[hot_aisle_patch, cold_aisle_patch, rack_patch, rack_patch_used])

### Visualisation de la répartition des types de GPU - Bar plot

Génère un graphique à barres illustrant la répartition des différents types de GPU pour une date spécifique, en utilisant les données fournies dans `df_selected_data` pour extraire les types et les quantités de GPU, puis trace un histogramme avec des barres colorées en bleu clair.

In [239]:
def plot_gpu_bar(ax: plt.Axes, selected_date: datetime.date, df_selected_data: pd.DataFrame) -> None:
    gpu_types = df_selected_data.iloc[0]['GPU_Types']
    gpu_names = list(gpu_types.keys())
    gpu_counts = list(gpu_types.values())
    
    ax.bar(gpu_names, gpu_counts, color='skyblue')
    ax.set_xlabel('GPU Types')
    ax.set_ylabel('Number of GPUs')
    ax.set_title(f'GPU Composition on {selected_date.strftime("%Y-%m")}')

### Fonction pour générer les deux graphiques

In [240]:
def plot_server_room(selected_date: str, num_racks: int) -> None:
    selected_date = pd.to_datetime(selected_date)
    
    df_selected_data = df_test_model[df_test_model['Date'] == selected_date]
    num_gpu = df_selected_data['Total_GPU_Count'].sum()
    dimensions = size_server_room(dict_variables_servers, num_gpu, num_racks)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10), gridspec_kw={'width_ratios': [3, 1]})
    plot_server_layout(ax1, selected_date, dict_variables_servers, num_racks, dimensions)
    plot_gpu_bar(ax2, selected_date, df_selected_data)
    
    total_area, total_area_used, percentage_used_area_used = area_used_servers(dimensions, num_racks)
    
    info_html = f"""
    <p><strong>Number of GPU:</strong> {num_gpu}</p>
    <p><strong>Total square meters:</strong> {total_area} m²</p>
    <p><strong>Square meters used:</strong> {total_area_used} m²</p>
    <p><strong>Percentage of total square meters that are being utilised:</strong> {percentage_used_area_used} %</p>
    """
    info_widget.value = info_html
    display(info_widget)
    
    plt.show()

Le widget **"Date"** (slider) permet de sélectionner un mois précis pour visualiser la répartition des GPU à ce moment-là. Ainsi, le widget **"Racks per Line"** (slider) permet de choisir le nombre de racks nécessaires par ligne, avec un minimum de deux lignes de racks obligatoires. 

Une synthèse concise de la surface occupée est affichée entre les widgets et les graphiques pour une meilleure compréhension de l'occupation de l'espace dans le datacenter.

In [241]:
widgets.interact(plot_server_room, selected_date=date_slider, num_racks=num_racks_slider);

interactive(children=(SelectionSlider(continuous_update=False, description='Date', layout=Layout(height='25px'…